In [1]:
import numpy as np
import pandas as pd
import networkx as nx
from collections import defaultdict
from matplotlib import pyplot as plt
import seaborn as sns
import mplcursors
import math

In [2]:
def init_graph(G, node_adj_frame):
  G.add_nodes_from([i for i in range(len(node_adj_frame))])
  labels = {}
  labels = node_adj_frame.columns
  for i in range(len(node_adj_frame)):
    snode= node_adj_frame[labels[0]][i]-1
    temp = node_adj_frame[labels[2]][i]
    if ',' in str(temp):
      sedge_arr=temp.split(',')
      for j in range(0,len(sedge_arr)):
        k=int(sedge_arr[j])
        if k<81:
          G.add_edge(snode,k-1)  
        else:
          G.add_edge(snode,k-2)
    elif temp==np.nan:
      print("ERROR:None found in th adjacency excel sheet")
    else :
      G.add_edge(snode,int(temp)-1)
  return

In [3]:
def init_graph_attr(G, AdjFile, df_IMR, df_MMR, IMR_Key, MMR_Key):
    node_adj_frame = pd.read_excel(AdjFile)
    node_list = node_adj_frame["KGISTalukN"].tolist()
    nodeAttr = {}
    init_graph(G, node_adj_frame)
    capability_vector = []
    capability_vector = [df_IMR[IMR_Key], df_MMR[MMR_Key]]
    node_attri_dict = {}
    for i, key in enumerate(df_IMR["Taluka"]):
        node_attri_dict[key] = [row[i] for row in capability_vector]
    for i in range(len(node_adj_frame)):
        temp = {}
        temp["capability"] = node_attri_dict.get(node_list[i], [0, 0])
        temp["nodeStress"] = 0
        temp["name"] = node_list[i]
        nodeAttr[i] = temp
    nx.set_node_attributes(G, nodeAttr)

In [4]:
G = nx.Graph()
df_IMR = pd.read_excel('PIA_Normalized_IMR.xlsx', sheet_name='Sheet1')
df_MMR = pd.read_excel('PIA_Normalized_MMR.xlsx', sheet_name='Sheet1')

In [5]:
dictionary_IMR={"Normalized IMR (ANC-20%)":"Stress (ANC-20%)","Normalized IMR (ANC-10%)":"Stress (ANC-10%)","Normalized IMR (ANC+10%)":"Stress (ANC+10%)","Normalized IMR (ANC+20%)":"Stress (ANC+20%)"}
dictionary_MMR = {"Normalized MMR (ANC -20%)":"Stress (ANC -20%)", "Normalized MMR (ANC -10%)":"Stress (ANC -10%)", "Normalized MMR (ANC +10%)":"Stress (ANC +10%)", "Normalized MMR (ANC +20%)":"Stress (ANC +20%)"}
matching_vector = [["Normalized IMR (ANC-20%)","Normalized IMR (ANC-10%)", "Normalized IMR (ANC+10%)", "Normalized IMR (ANC+20%)"], ["Normalized MMR (ANC -20%)", "Normalized MMR (ANC -10%)", "Normalized MMR (ANC +10%)", "Normalized MMR (ANC +20%)"]]

In [6]:
dist_taluk_dict = defaultdict(list)
for k, v in zip(df_IMR["District_GIS"],df_IMR["Taluka"]):
  dist_taluk_dict[k].append(v)

In [7]:
IMR_Key = 'Normalized IMR'
MMR_Key = 'Normalized MMR'

In [8]:
init_graph_attr(G, 'IMR_Stress_AdjFile.xlsx', df_IMR, df_MMR, IMR_Key, MMR_Key)

In [ ]:
for n in G.nodes():
    print(G.nodes[n])

In [15]:
def get_node_stress(G):
  taluk_stress_dict={}
  for n in G.nodes():
    centroid = [0, 0]
    temp_dist_imr = 0
    temp_dist_mmr = 0
    temp_dist = 0
    node_stress = 0
    neigList = list(G.neighbors(n))
    for nei in neigList:
      try:
        temp_dist_imr += G.nodes[nei]['capability'][0]
        temp_dist_mmr += G.nodes[nei]['capability'][1]
        # temp_dist_imr = abs(G.nodes[nei]['capability'][0] - G.nodes[n]['capability'][0])
        # temp_dist_mmr = abs(G.nodes[nei]['capability'][1] - G.nodes[n]['capability'][1])
        # temp_dist = math.sqrt(abs(temp_dist_imr + temp_dist_mmr))
        # node_stress += temp_dist
      except(KeyError):
        pass
    centroid = [temp_dist_imr/len(neigList), temp_dist_mmr/len(neigList)]
    # print(centroid)
    temp_dist_mmr = 0
    temp_dist_imr = 0
    # print(G.nodes[n]['capability'][0])
    # temp_dist_imr = (centroid[0] - G.nodes[n]['capability'][0])**2
    # temp_dist_mmr = (centroid[1] - G.nodes[n]['capability'][1])**2
    # temp_dist = math.sqrt(temp_dist_imr + temp_dist_mmr)
    # node_stress += temp_dist
    for nei in neigList:  
      try:
        temp_dist_imr = (centroid[0] - G.nodes[n]['capability'][0])**2
        temp_dist_mmr = (centroid[1] - G.nodes[n]['capability'][1])**2
        temp_dist = math.sqrt(temp_dist_imr + temp_dist_mmr)
        node_stress = temp_dist
      except(KeyError):
        pass  
    try:
      G.nodes[n]['nodeStress'] = node_stress
    except(KeyError):
      # CHANGE THIS, IT IS WRONG
      G.nodes[n]['nodeStress'] = node_stress
      # G.nodes[n]["nodeStress"]  = math.sqrt((l2_sum_of_sq/len(neigList))**2)
    try:
      taluk_stress_dict[G.nodes[n]["name"]]=G.nodes[n]["nodeStress"]
    except(KeyError):
      pass
  return taluk_stress_dict

In [16]:
stress = {}
for key_IMR, key_MMR in zip(dictionary_IMR.keys(), dictionary_MMR.keys()):    
    init_graph_attr(G,'IMR_Stress_AdjFile.xlsx',df_IMR, df_MMR, key_IMR, key_MMR)
    stress=get_node_stress(G)
    print(stress)
    df_IMR[dictionary_IMR[key_IMR]]=df_IMR['Taluka'].map(stress)
    df_MMR[dictionary_MMR[key_MMR]]=df_MMR['Taluka'].map(stress)

{'Chikkodi': 0.1300138779350256, 'Athani': 0.3569499263908474, 'Raibag': 0.2778846929271669, 'Gokak': 0.17872166892984206, 'Hukkeri': 0.11015756307367092, 'Belagavi': 0.08422554611599857, 'Khanapur': 0.346053349419314, 'Savadatti': 0.13844047429855302, 'Bailhongal': 0.23671924413614004, 'Ramadurg': 0.2635263885974776, 'Kitthuru': 0.03250422687237802, 'Nippani': 0.061243093513823336, 'Kagavada': 0.31280301676508393, 'Mudalagi': 0.3652102095481724, 'Jamakhandi': 0.32801982511086164, 'Bilagi': 0.20724359599870243, 'Mudhol': 0.26115424624321537, 'Badami': 0.15452797273615598, 'Bagalkote': 0.12141427095005429, 'Hungund': 0.2203585859433877, 'Guledagudda': 0.06582245063569041, 'Ilkal': 0.2659919679506544, 'Rabakavi_banahatti': 1.1599849323253262, 'Bijapur': 0.42679208605098673, 'Indi': 0.2084291668736599, 'Sindagi': 0.32663929593707525, 'Basavan_bagewadi': 0.9553380112499831, 'Muddebihal': 0.24572105492319335, 'Nidagundi': 0.12437101439121055, 'Babaleshwara': 0.3884241535530969, 'Chadachana'

In [13]:
for n in G.nodes():
    print(G.nodes[n])

{'capability': [0.886443449571792, 0.874193125225058], 'nodeStress': 0.025794860382524272, 'name': 'Chikkodi'}
{'capability': [0.913773040146328, 0.367823297643739], 'nodeStress': 0.0700652367003379, 'name': 'Athani'}
{'capability': [0.934660071448362, 0.788675344444311], 'nodeStress': 0.04585495095412351, 'name': 'Raibag'}
{'capability': [0.960836218147914, 0.916381851713975], 'nodeStress': 0.021792696248854708, 'name': 'Gokak'}
{'capability': [0.921755018222572, 0.788675708824945], 'nodeStress': 0.026058809486583942, 'name': 'Hukkeri'}
{'capability': [0.638553402283893, 0.736018818507573], 'nodeStress': 0.016283992749611457, 'name': 'Belagavi'}
{'capability': [0.223242228945142, 0.788703939469489], 'nodeStress': 0.03913687774096819, 'name': 'Khanapur'}
{'capability': [0.901733399894342, 0.782649385267076], 'nodeStress': 0.022608500147807676, 'name': 'Savadatti'}
{'capability': [0.577324992448355, 0.827524414006224], 'nodeStress': 0.04825683794397002, 'name': 'Bailhongal'}
{'capabilit

In [14]:
df_IMR

,District_GIS,Taluka,Infant Mortality Rate,Normalized IMR,Mothers who haven't had at least 4 antenatal care visits(%),Predicted Change In IMR (ANC -20%),New IMR (ANC-20%),Normalized IMR (ANC-20%),Impact Score (ANC -20%),Predicted Change In IMR (ANC -10%),...,Normalized IMR (Vitamin A+10%),Impact Score (Vitamin A +10%),Predicted Change In IMR (Vitamin A +20%),New IMR (Vitamin A+20%),Normalized IMR (Vitamin A+20%),Impact Score (Vitamin A +20%),Stress (ANC-20%),Stress (ANC-10%),Stress (ANC+10%),Stress (ANC+20%)
0,Kalburgi,Afzalpur,5.558428,0.942582,20.00355,-1.440256,4.118172,0.944348,0.490016,-0.720128,...,0.940914,0.673002,2.916155,8.474583,0.936699,0.673002,0.028286,0.028706,0.029327,0.029584
1,Chikkamagaluru,Ajjampura,13.672259,0.858767,15.92314,-1.146466,12.525793,0.858248,0.387197,-0.573233,...,0.858609,0.770052,2.569192,16.241451,0.856133,0.770052,0.021372,0.021385,0.021412,0.021427
2,Kalburgi,Aland,10.468411,0.891863,27.93242,-2.011134,8.457277,0.899912,0.689810,-1.005567,...,0.893266,0.848179,2.289878,12.758289,0.892264,0.848179,0.027867,0.027692,0.027177,0.026885
3,Dharwad,Alnavara,39.509821,0.591869,15.17857,-1.092857,38.416964,0.593104,0.368436,-0.546429,...,0.585644,0.492161,3.562686,43.072507,0.577813,0.492161,0.024812,0.024449,0.023541,0.023051
4,Hassan,Alur,36.381576,0.624183,10.93930,-0.787630,35.593947,0.622014,0.261613,-0.393815,...,0.622799,0.747382,2.650238,39.031815,0.619727,0.747382,0.022860,0.022853,0.022752,0.022682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,Kalburgi,Yadrami,33.755352,0.651312,29.57785,-2.129605,31.625746,0.662651,0.731272,-1.064803,...,0.642831,0.359374,4.037415,37.792767,0.632580,0.359374,0.039539,0.039490,0.039420,0.039400
222,Chamarajanagara,Yalandur,18.631343,0.807541,14.56877,-1.048951,17.582391,0.806465,0.353070,-0.524476,...,0.800602,0.413616,3.843494,22.474837,0.791474,0.413616,0.063484,0.063451,0.063264,0.063142
223,Bengaluru_Urban,Yelahanka,45.156309,0.533541,3.09884,-0.223116,44.933192,0.526373,0.064048,-0.111558,...,0.525359,0.397332,3.901712,49.058021,0.515725,0.397332,0.067027,0.067021,0.067009,0.067003
224,Koppal,Yelburga,26.639909,0.724813,35.73127,-2.572651,24.067257,0.740055,0.886327,-1.286326,...,0.712902,0.160607,4.748033,31.387942,0.699018,0.160607,0.042544,0.042372,0.041779,0.041428


In [17]:
df_IMR.to_excel("NewStress_PIA_New_IMR.xlsx")
df_MMR.to_excel("NewStress_PIA_New_MMR.xlsx")

In [18]:
def aggregate(taluk_stress_dict):
  dist_stress={}
  for dist,taluks in dist_taluk_dict.items():
    # print(dist)
    # print(taluks)
    agg_stress=0
    for taluk in taluks:
      try:
        agg_stress += taluk_stress_dict[taluk]
      except(KeyError):
        pass
    dist_stress[dist]=agg_stress/len(taluks)
  return dist_stress

In [19]:
result=aggregate(stress)
print(result)

{'Kalburgi': 0.2080666275511935, 'Chikkamagaluru': 0.15684600506887006, 'Dharwad': 0.2668612227814122, 'Hassan': 0.3306658747310266, 'Bengaluru_Urban': 0.1902490627592411, 'Uttara_Kannada': 0.269524141293004, 'Belagavi': 0.2049546171927629, 'Bidar': 0.1311000565325688, 'Vijayapura': 0.202199837456801, 'Bagalkot': 0.18325451342652677, 'Chikkaballapura': 0.2405443946669765, 'Ballari': 0.22350297882656253, 'Kolara': 0.32357170603931135, 'Dakshina_Kannada': 0.13246373349237242, 'Shivamogga': 0.17072555415580384, 'Udupi': 0.15132783597625016, 'Haveri': 0.17880163958287904, 'Chitradurga': 0.17280817313761587, 'Chamarajanagara': 0.26606818552855127, 'Davanagere': 0.35455539369230743, 'Ramanagara': 0.28655798759167905, 'Tumakuru': 0.152416361669417, 'Bengaluru_Rural': 0.3142888076501897, 'Raichur': 0.24152018320659027, 'Gadag': 0.21337825022619136, 'Koppal': 0.2657134971799521, 'Yadgir': 0.4264405668165738, 'Mysuru': 0.30155643498129914, 'Mandya': 0.20403286381255328, 'Kodagu': 0.2119306649319